In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import json
import glob

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

In [2]:
echonet_dir = '/zfs/wficai/Data/echonet_data/EchoNet-Dynamic/'
frame_export_dir = '/zfs/wficai/Data/echonet_data/frames/'
split_csv_export_dir = '/zfs/wficai/Data/echonet_data/'
os.chdir(echonet_dir)
d = pd.read_csv('FileList.csv')

In [3]:
res_x = 112
res_y = 112

In [4]:
d['vid_path'] = d.FileName.apply(lambda x: f"{echonet_dir}/Videos/{x}.avi")

In [5]:
d.head()

,FileName,EF,ESV,EDV,FrameHeight,FrameWidth,FPS,NumberOfFrames,Split,vid_path
0,0X100009310A3BD7FC,78.498406,14.881368,69.210534,112,112,50,174,VAL,/zfs/wficai/Data/echonet_data/EchoNet-Dynamic/...
1,0X1002E8FBACD08477,59.101988,40.383876,98.742884,112,112,50,215,TRAIN,/zfs/wficai/Data/echonet_data/EchoNet-Dynamic/...
2,0X1005D03EED19C65B,62.363798,14.267784,37.909734,112,112,50,104,TRAIN,/zfs/wficai/Data/echonet_data/EchoNet-Dynamic/...
3,0X10075961BC11C88E,54.545097,33.143084,72.914210,112,112,55,122,TRAIN,/zfs/wficai/Data/echonet_data/EchoNet-Dynamic/...
4,0X10094BA0A028EAC3,24.887742,127.581945,169.855024,112,112,52,207,VAL,/zfs/wficai/Data/echonet_data/EchoNet-Dynamic/...


In [16]:
# standardize ESV and EDV
quantile_90 = pd.concat([d.ESV, d.EDV]).quantile(0.90)
quantile_90

123.84948384000015

In [17]:
d['ESV_scaled'] = d['ESV']/quantile_90
d['EDV_scaled'] = d['EDV']/quantile_90

In [23]:
for s in ('TRAIN', 'VAL', 'TEST'):
    d.query("Split==@s").to_csv(f"{split_csv_export_dir}/{s}.csv", index=None)

# Preprocess frames

In [7]:
def preproc(frames):

    frames = frames.permute(0,3,1,2)
    
    #Resize
    splitFrames = torch.split(frames, 10)
    resizedFrames = []
    aspectRatio = (frames.shape[3])/(frames.shape[2])
    for f in splitFrames:
        f = TF.resize(f, res_y)
        f = TF.center_crop(f, (res_x, res_y))
        
        resizedFrames.append(f)

    resizedFrames = torch.concat(resizedFrames)
    
    return resizedFrames

In [8]:
frame_list = []
for ix, row in d.iterrows():
    print(" "*100, end='\r')
    print(f"Converting video {ix+1} of {len(d)}.", end="\r")
    
    frames, _, _ =  torchvision.io.read_video(row['vid_path'], pts_unit = 'sec')
    frames = preproc(frames)
    
    # save individual frames
    for ix, f in enumerate(frames):
        filename = f"{frame_export_dir}{row['FileName']}_frame{str(ix).zfill(4)}.jpg"
        frame_list.append({'filename': row['FileName'], 'frame': ix, 'jpg_path': filename})
        torchvision.io.write_jpeg(f, filename)      

In [10]:
df_frames = pd.DataFrame(frame_list)
df_frames = df_frames.rename(columns={'filename': 'FileName'})
df_frames.to_csv(f"{split_csv_export_dir}/frames.csv", index=None)

In [11]:
df_frames

,FileName,frame,jpg_path
0,0X100009310A3BD7FC,0,/zfs/wficai/Data/echonet_data/frames/0X1000093...
1,0X100009310A3BD7FC,1,/zfs/wficai/Data/echonet_data/frames/0X1000093...
2,0X100009310A3BD7FC,2,/zfs/wficai/Data/echonet_data/frames/0X1000093...
3,0X100009310A3BD7FC,3,/zfs/wficai/Data/echonet_data/frames/0X1000093...
4,0X100009310A3BD7FC,4,/zfs/wficai/Data/echonet_data/frames/0X1000093...
...,...,...,...
1770631,0X5515B0BD077BE68A,121,/zfs/wficai/Data/echonet_data/frames/0X5515B0B...
1770632,0X5515B0BD077BE68A,122,/zfs/wficai/Data/echonet_data/frames/0X5515B0B...
1770633,0X5515B0BD077BE68A,123,/zfs/wficai/Data/echonet_data/frames/0X5515B0B...
1770634,0X5515B0BD077BE68A,124,/zfs/wficai/Data/echonet_data/frames/0X5515B0B...
